In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import os
import gpustat

stats = gpustat.GPUStatCollection.new_query()
ids = map(lambda gpu: int(gpu.entry['index']), stats)
ratios = map(lambda gpu: float(gpu.entry['memory.used'])/float(gpu.entry['memory.total']), stats)
bestGPU = min(zip(ids, ratios), key=lambda x: x[1])[0]

print("setGPU: Setting GPU to: {}".format(bestGPU))
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = str(bestGPU)

setGPU: Setting GPU to: 1


**Après l'entretien:**

# MODEL:
1 model normal
1 model Learning rate /2 regulierement qui apporte une meilleure generalisation que directement un petit learning rate
1 model avec weights decay #L² (qui ameliore nettement l'orthogonalisation et donc le generalisation pour les SGD #Simon'sArticle)

(+effets positifs des petits minibatch sur la generalisation)

# WEIGTHS
Tester les deltas de poids pour une couche, mais aussi pour qq neurones seuls.
Normaliser par rapport aux poids initiaux ou finaux.

Regarder les poids qui changent le plus pour un neurone, et faire un histogramme pour plusieurs neurones. Voir si il y a un pique, ou plusieurs lorsqu'on generalise mieux ou pas.

w_0 * w_t ~= 0 si orthogonaux (donc plus de chance de bien generaliser car tous les poids ont bougés de leur position initiale)

# ACTIVATIONS
Regarder l'information mutuelle AVEC DES INT #binaires (! restriction d'arrondi)
Regarder la métrique spéciale de l'article pour comparer ce qu'apporte un neurone par rapport à tous les autres neurones de la couche (! restriction linéaire)


### Import packages

In [4]:
# IMPORT
import numpy

import keras
from keras import optimizers
from keras.datasets import cifar10 # we can use also cifar100
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

import matplotlib.pyplot as plt

import sklearn
import sklearn.metrics

#print(sklearn.__version__)

In [5]:
# LOAD DATA
num_classes = 10
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [38]:
# MODEL 0
def model_0():
  # create model
  # https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  # Optimizer
  sgd = optimizers.SGD(lr=0.01)
  #opt = keras.optimizers.rmsprop(lr=0.001, decay=0.0)
  # Compile model
  #loss : 'categorical_crossentropy', 'mean_squared_error'
  #optimizer : 'adam', 'sgd',adadelta, adagrad, RMSprop, Adamax, Nadam
  model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

In [39]:
# MODEL 1
def model_1():
  # create model
  # https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  # Optimizer
  sgd = optimizers.SGD(lr=0.01)
  #opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
  # Compile model
  #loss : 'categorical_crossentropy', 'mean_squared_error'
  #optimizer : 'adam', 'sgd',adadelta, adagrad, RMSprop, Adamax, Nadam
  model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

In [40]:
# MODEL 2
def model_2():
  # create model
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  # Optimizer
  sgd = optimizers.SGD(lr=0.02)
  #opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
  # Compile model
  #loss : 'categorical_crossentropy', 'mean_squared_error'
  #optimizer : 'adam', 'sgd',adadelta, adagrad, RMSprop, Adamax, Nadam
  model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

In [41]:
# TRAINING
batch_size = 32
epochs = 20
# build the models
model0 = model_0()
model1 = model_1()
model2 = model_2()
# get weights
weights01 = model0.get_weights()
weights11 = model1.get_weights()
weights21 = model2.get_weights()

In [31]:
# Fit the model 0
data_augmentation = False
if not data_augmentation:
    print('Not using data augmentation.')
    model0.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)

Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 12s 243us/step - loss: 0.5821 - acc: 0.8018 - val_loss: 0.7067 - val_acc: 0.7621
Epoch 2/20
50000/50000 [==============================] - 12s 241us/step - loss: 0.5952 - acc: 0.7977 - val_loss: 0.7284 - val_acc: 0.7659
Epoch 3/20
50000/50000 [==============================] - 12s 241us/step - loss: 0.5896 - acc: 0.7995 - val_loss: 0.7054 - val_acc: 0.7719
Epoch 4/20
50000/50000 [==============================] - 12s 242us/step - loss: 0.5803 - acc: 0.8035 - val_loss: 0.7159 - val_acc: 0.7749
Epoch 5/20
50000/50000 [==============================] - 12s 242us/step - loss: 0.5705 - acc: 0.8083 - val_loss: 0.7361 - val_acc: 0.7666
Epoch 6/20
50000/50000 [==============================] - 12s 242us/step - loss: 0.5754 - acc: 0.8065 - val_loss: 0.7108 - val_acc: 0.7717
Epoch 7/20
50000/50000 [==============================] - 12s 240us/step - loss: 0.5849

In [42]:
# Fit the model 1
if not data_augmentation:
    print('Not using data augmentation.')
    model1.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)

Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 12s 245us/step - loss: 2.1291 - acc: 0.2075 - val_loss: 1.9704 - val_acc: 0.3017
Epoch 2/20
50000/50000 [==============================] - 12s 236us/step - loss: 1.8232 - acc: 0.3382 - val_loss: 1.6276 - val_acc: 0.4162
Epoch 3/20
50000/50000 [==============================] - 12s 236us/step - loss: 1.6410 - acc: 0.4019 - val_loss: 1.5283 - val_acc: 0.4377
Epoch 4/20
50000/50000 [==============================] - 12s 237us/step - loss: 1.5233 - acc: 0.4469 - val_loss: 1.4398 - val_acc: 0.4875
Epoch 5/20
50000/50000 [==============================] - 12s 238us/step - loss: 1.4283 - acc: 0.4811 - val_loss: 1.3504 - val_acc: 0.5109
Epoch 6/20
50000/50000 [==============================] - 12s 237us/step - loss: 1.3554 - acc: 0.5117 - val_loss: 1.2507 - val_acc: 0.5604
Epoch 7/20
50000/50000 [==============================] - 12s 236us/step - loss: 1.2930

In [55]:
from keras.callbacks import LearningRateScheduler
from keras import backend as K

In [56]:
def scheduler(epoch):
    if epoch == 10:
        print("Changing learning rate")
        K.set_value(model2.optimizer.lr, 0.01)
    return K.get_value(model2.optimizer.lr)

In [57]:
change_lr = LearningRateScheduler(scheduler)

In [58]:
# Fit the model 2
if not data_augmentation:
    print('Not using data augmentation.')
    model2.fit(x_train, y_train, 
               batch_size=batch_size, 
               epochs=epochs, 
               validation_data=(x_test, y_test), 
               shuffle=True,
               callbacks=[change_lr])

Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 12s 244us/step - loss: 2.0052 - acc: 0.2627 - val_loss: 1.6887 - val_acc: 0.3953
Epoch 2/20
50000/50000 [==============================] - 12s 235us/step - loss: 1.6077 - acc: 0.4180 - val_loss: 1.5332 - val_acc: 0.4609
Epoch 3/20
50000/50000 [==============================] - 12s 238us/step - loss: 1.4024 - acc: 0.4932 - val_loss: 1.2447 - val_acc: 0.5524
Epoch 4/20
50000/50000 [==============================] - 12s 235us/step - loss: 1.2772 - acc: 0.5434 - val_loss: 1.1912 - val_acc: 0.5816
Epoch 5/20
50000/50000 [==============================] - 12s 236us/step - loss: 1.1928 - acc: 0.5759 - val_loss: 1.0623 - val_acc: 0.6271
Epoch 6/20
50000/50000 [==============================] - 12s 236us/step - loss: 1.1162 - acc: 0.6036 - val_loss: 1.0113 - val_acc: 0.6410
Epoch 7/20
50000/50000 [==============================] - 12s 237us/step - loss: 1.0538

In [59]:
# EVALUATION
# Final evaluation of the models
scores0 = model0.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores0[0])
print('Test accuracy:', scores0[1])
print("Model 0 Error: %.2f%%" % (100-scores0[1]*100))
scores1 = model1.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores1[0])
print('Test accuracy:', scores1[1])
print("Model 1 CNN Error: %.2f%%" % (100-scores1[1]*100))
scores2 = model2.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores2[0])
print('Test accuracy:', scores2[1])
print("Model 2 Error: %.2f%%" % (100-scores2[1]*100))

10000/10000 [==============================] - 1s 110us/step
Test loss: 2.3054959476470946
Test accuracy: 0.1032
Model 0 Error: 89.68%
10000/10000 [==============================] - 1s 95us/step
Test loss: 0.8478721293449402
Test accuracy: 0.7023
Model 1 CNN Error: 29.77%
10000/10000 [==============================] - 1s 96us/step
Test loss: 0.6642614018917083
Test accuracy: 0.7687
Model 2 Error: 23.13%


# WEIGHTS

In [27]:
#https://keras.io/layers/about-keras-layers/
#print(weights)
#print(weights[0][0][0][0])
# print(weights12[10])
#print(weights[11]) # last layer

weights02 = model0.get_weights()
weights12 = model1.get_weights()
weights22 = model2.get_weights()

weights02[0].shape

(3, 3, 3, 32)

In [1]:
# NORMALISATION
from numpy import linalg

COUCHE = 8

D_W0 = numpy.absolute(weights01[COUCHE]-weights02[COUCHE])
D_W1 = numpy.absolute(weights11[COUCHE]-weights12[COUCHE])
D_W2 = numpy.absolute(weights21[COUCHE]-weights22[COUCHE])

D_W0_norm = linalg.norm(D_W0)
D_W1_norm = linalg.norm(D_W1) # ,np.inf)
D_W2_norm = linalg.norm(D_W2)


D_W0_normed = D_W0/D_W0_norm
D_W1_normed = D_W1/D_W1_norm
D_W2_normed = D_W2/D_W2_norm

# another norm
#weights11_norm2 = (weights11 - weights11.mean()) / (weights11.max() - weights11.min())
#weights12_norm2 = (weights12 - weights12.mean()) / (weights12.max() - weights12.min())
#weights21_norm2 = (weights21 - weights21.mean()) / (weights21.max() - weights21.min())
#weights22_norm2 = (weights22 - weights22.mean()) / (weights22.max() - weights22.min())

NameError: name 'numpy' is not defined

In [0]:
# PLOT
# model0
plt.hist(D_W0_normed, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram D_W0_normed with 'auto' bins")
plt.show()

NameError: ignored

In [0]:
#model1
plt.hist(D_W1_normed, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram D_W1_normed with 'auto' bins")
plt.show()

In [0]:
#model2
plt.hist(D_W2_normed, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram D_W2_normed with 'auto' bins")
plt.show()

# ACTIVATIONs

In [0]:
!pip install -q keract 
from keract import get_activations

In [0]:
# pour le test set, on regarde l'activation d'un echantillon de pairs de neurones (creation de X et Y)
IM0 = get_activations(model0, [x_test[0], x_test[1], x_test[2], x_test[3], x_test[4], x_test[5], x_test[6], x_test[7], x_test[8], x_test[9], x_test[10], x_test[11], x_test[12], x_test[13]])
print(IM0.keys())
#IM1 = get_activations(model1, x_test)
#IM2 = get_activations(model2, x_test)

In [0]:
#nameOfTheCouche = 'flatten_13/Reshape:0'
nameOfTheCouche = 'dense_25/BiasAdd:0'
numActivations = len(IM0[nameOfTheCouche])
print('number of data used for activation : '+str(numActivations))
numNeurons = len(IM0[nameOfTheCouche][0])
print('number of neurons for the layer: '+str(numNeurons))
#print(IM0['dense_2/BiasAdd:0'])
plt.imshow(IM0[nameOfTheCouche])
plt.show()

In [0]:
# on selectionne des pair de neurones
NombrePairs = 100
NeuronsSelected = numpy.zeros((NombrePairs,2))
for i in range(NombrePairs):
  NeuronsSelected[i] = numpy.ceil(numpy.random.rand(2)*numNeurons)
  while NeuronsSelected[i][0] == NeuronsSelected[i][1]:
    NeuronsSelected[i] = numpy.ceil(numpy.random.rand(2)*numNeurons)
#print(NeuronsSelected[56])
#print(NeuronsSelected[56][0])

In [0]:
MutualInfo = numpy.zeros((NombrePairs,1))
for j in range(NombrePairs):
  X = numpy.zeros((numActivations,1))
  Y = numpy.zeros((numActivations,1))

  i = 0
  for activation in IM0[nameOfTheCouche]:
    #print(activation)

    X[i] = activation[int(NeuronsSelected[j][0])-1]
    Y[i] = activation[int(NeuronsSelected[j][1])-1]
    #print(X[i])
    i = i+1

  X = X.flatten()
  Y = Y.flatten()
  #print(X)
  #print(Y)
  MutualInfo[j] = sklearn.metrics.normalized_mutual_info_score(X,Y)
  #print(MutualInfo[j])


In [0]:
print(MutualInfo)

In [28]:
sklearn.metrics.normalized_mutual_info_score([1.1,1.0,1.0,1.0,0.0],[1.1,0.1,0.1,1.1,0.1])


/linux/mnovak/Documents/Thesis/Code/myenv/lib/python3.5/site-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


0.36400889786344465